In [14]:
from os import name
import pandas as pd
import numpy as np

pd.set_option("display.width", 140)
pd.set_option("display.max_rows", 30)

# Mock "Orders"
orders = pd.DataFrame({
    "InvoiceNo": ["10001","10002","10003","10004"],
    "CustomerID": [111, 222, 111, 333],
    "InvoiceDate": pd.to_datetime(["2011-01-01","2011-01-02","2011-02-05","2011-02-20"]),
    "Country": ["UK","France","UK","Germany"]
})

# Mock "Order Items"
order_items = pd.DataFrame({
    "InvoiceNo": ["10001","10001","10002","10003","10003","10004"],
    "StockCode": ["A1","B2","A1","C3","A1","B2"],
    "Quantity": [2,1,3,1,1,2],
    "UnitPrice": [5.0,15.0,5.0,10.0,5.0,15.0]
})

# Mock "Products"
products = pd.DataFrame({
    "StockCode": ["A1","B2","C3"],
    "Description": ["Mug","T-shirt","Hat"],
    "Category": ["Home","Clothes","Accessories"]
})


df = (order_items
      .merge(orders,on="InvoiceNo", how="inner")
      .merge(products,on="StockCode",how="left"))
df["Revenue"] = df['Quantity'] * df["UnitPrice"]
df["Month"] = df["InvoiceDate"].dt.to_period("M")

print(df.head(10))

pivot = pd.pivot_table(
    df,
    values="Revenue",
    index="Month",
    columns="Category",
    aggfunc="sum",
    fill_value=0,
    margins=True,          # adds a “Total” column/row
    margins_name="Total"
)

print(pivot)

melted = (pivot.
          reset_index()
          .melt(id_vars="Month",var_name="Category",value_name="Revenue")
          )
melted = melted[melted["Category"] != "Total"]
print(melted)

top2Categories = (df.groupby(["Month","Category"])["Revenue"].sum().reset_index(name="Top2Revenue")
                  .sort_values(["Month","Top2Revenue"], ascending=[True,False])
                  .groupby("Month",group_keys=False)
                  .head(2))

print(top2Categories)

top3Costumers = (df.groupby(["Country","CustomerID"])["Revenue"].sum().reset_index(name="TotalRevenue")
                .sort_values(["Country","TotalRevenue"], ascending=[True,False])
                .groupby("Country", group_keys=False)
                .head(3))

print(top3Costumers)










  InvoiceNo StockCode  Quantity  UnitPrice  CustomerID InvoiceDate  Country Description     Category  Revenue    Month
0     10001        A1         2        5.0         111  2011-01-01       UK         Mug         Home     10.0  2011-01
1     10001        B2         1       15.0         111  2011-01-01       UK     T-shirt      Clothes     15.0  2011-01
2     10002        A1         3        5.0         222  2011-01-02   France         Mug         Home     15.0  2011-01
3     10003        C3         1       10.0         111  2011-02-05       UK         Hat  Accessories     10.0  2011-02
4     10003        A1         1        5.0         111  2011-02-05       UK         Mug         Home      5.0  2011-02
5     10004        B2         2       15.0         333  2011-02-20  Germany     T-shirt      Clothes     30.0  2011-02
Category  Accessories  Clothes  Home  Total
Month                                      
2011-01           0.0     15.0  25.0   40.0
2011-02          10.0     30.0   5.